필수 라이브러리 설치

In [ ]:
pip install requests pandas matplotlib schedule


기본 아이템 DB 제작 코드

In [ ]:
import sqlite3

# 데이터베이스 연결 (파일이 없으면 새로 생성됩니다)
conn = sqlite3.connect('item_price_data.db')
cursor = conn.cursor()

# item_names 테이블 생성 (이미 존재할 경우 오류 방지를 위해 IF NOT EXISTS 사용)
cursor.execute('''
    CREATE TABLE IF NOT EXISTS item_names (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        Name TEXT UNIQUE NOT NULL
    )
''')

# 추가할 아이템 이름 리스트
item_names = [
    '아드레날린 각인서', '원한 각인서', '예리한 둔기 각인서',
    '돌격대장 각인서', '저주받은 인형 각인서', '기습의 대가 각인서',
    '타격의 대가 각인서', '질량 증가 각인서', '마나의 흐름 각인서',
    '슈퍼 차지 각인서'
]

# 아이템 이름을 테이블에 추가
for name in item_names:
    try:
        cursor.execute("INSERT OR IGNORE INTO item_names (Name) VALUES (?)", (name,))
    except sqlite3.Error as e:
        print(f"Error occurred: {e}")

# 데이터베이스에 변경사항 저장
conn.commit()
print("아이템 이름이 item_names 테이블에 저장되었습니다.")

# 테이블에 저장된 모든 아이템 이름 확인
cursor.execute("SELECT * FROM item_names")
all_items = cursor.fetchall()
print("저장된 아이템 이름 목록:")
for item in all_items:
    print(item)

# 연결 종료
conn.close()


아이템 이름이 item_names 테이블에 저장되었습니다.
저장된 아이템 이름 목록:
(1, '아드레날린 각인서')
(2, '원한 각인서')
(3, '예리한 둔기 각인서')
(4, '돌격대장 각인서')
(5, '저주받은 인형 각인서')
(6, '기습의 대가 각인서')
(7, '타격의 대가 각인서')
(8, '질량 증가 각인서')
(9, '마나의 흐름 각인서')
(10, '슈퍼 차지 각인서')


한글 폰트 적용 후 런타임 재시작

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 1s (9,091 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 123629 files and dire

메인 코드

In [ ]:
import requests
import pandas as pd
import sqlite3
from datetime import datetime
import matplotlib.pyplot as plt
import time

plt.rc('font', family='NanumBarunGothic')

# API 호출을 위한 정보
api_key = ""  # 본인의 API 키로 교체
item_search_url = "https://developer-lostark.game.onstove.com/markets/items"
headers = {
    "accept": "application/json",
    "authorization": f"bearer {api_key}",
    "Content-Type": "application/json"
}

# DB 파일명
db_filename = 'item_price_data.db'

def fetch_and_save_data():
    # DB에서 아이템 이름 조회
    conn = sqlite3.connect(db_filename)
    item_names_df = pd.read_sql_query("SELECT DISTINCT Name FROM item_names", conn)
    item_names = item_names_df["Name"].tolist()

    # 시세 조회 및 데이터 저장
    for item_name in item_names:
        data = {
            "Sort": "GRADE",
            "CategoryCode": "40000",
            "CharacterClass": "",
            "ItemTier": "",
            "ItemGrade": "유물",
            "ItemName": item_name,
            "PageNo": 1,
            "SortCondition": "ASC"
        }

        response = requests.post(item_search_url, headers=headers, json=data)

        if response.status_code == 200:
            market_data = response.json()
            if 'Items' in market_data and market_data['Items']:
                df = pd.DataFrame(market_data['Items'])
                today = datetime.now()
                df['Date'] = today
                df.to_sql('item_prices', conn, if_exists='append', index=False)
                print(f"{item_name} 데이터 저장 완료!")
            else:
                print(f"'{item_name}'에 해당하는 아이템을 찾을 수 없습니다.")
        else:
            print(f"Error: {response.status_code} - {response.text} ({item_name})")

    print("모든 데이터가 데이터베이스에 저장되었습니다.")

    # DB에서 모든 아이템의 데이터 불러오기 및 그래프 그리기
    query = "SELECT * FROM item_prices"
    data_from_db = pd.read_sql_query(query, conn)

    if not data_from_db.empty:
        plt.figure(figsize=(14, 8))

        for item_name in item_names:
            item_data = data_from_db[data_from_db["Name"] == item_name]
            plt.plot(item_data['Date'], item_data['RecentPrice'], marker='o', label=item_name)

        plt.title("각 아이템의 최근 가격 변동 추이")
        plt.ylabel("최근 가격 (Recent Price)")
        plt.legend(loc='upper left', bbox_to_anchor=(1, 1))  # 범례 위치를 그래프 밖으로 설정
        plt.grid(True)
        plt.tight_layout()
        plt.xticks([])  # x축 범례 숨김
        plt.show()
    else:
        print("데이터베이스에 데이터가 없습니다.")

    conn.close()

# 무한 루프를 통해 10분 간격으로 데이터 가져오기 및 저장
while True:
    fetch_and_save_data()  # 데이터 가져오기 및 저장
    time.sleep(600)  # 10분 대기 (600초)



[START] 데이터 업데이트 작업 시작
[INFO] item_prices 테이블 확인 및 생성 완료.
[INFO] DB에서 조회된 아이템 이름: ['아드레날린 각인서', '원한 각인서', '예리한 둔기 각인서', '돌격대장 각인서', '저주받은 인형 각인서', '기습의 대가 각인서', '타격의 대가 각인서', '질량 증가 각인서', '마나의 흐름 각인서', '슈퍼 차지 각인서']
[INFO] '아드레날린 각인서' 시세 데이터 조회 중...
[SUCCESS] '아드레날린 각인서' 데이터 저장 완료!
[INFO] '원한 각인서' 시세 데이터 조회 중...
[SUCCESS] '원한 각인서' 데이터 저장 완료!
[INFO] '예리한 둔기 각인서' 시세 데이터 조회 중...
[SUCCESS] '예리한 둔기 각인서' 데이터 저장 완료!
[INFO] '돌격대장 각인서' 시세 데이터 조회 중...
[SUCCESS] '돌격대장 각인서' 데이터 저장 완료!
[INFO] '저주받은 인형 각인서' 시세 데이터 조회 중...
[SUCCESS] '저주받은 인형 각인서' 데이터 저장 완료!
[INFO] '기습의 대가 각인서' 시세 데이터 조회 중...
[SUCCESS] '기습의 대가 각인서' 데이터 저장 완료!
[INFO] '타격의 대가 각인서' 시세 데이터 조회 중...
[SUCCESS] '타격의 대가 각인서' 데이터 저장 완료!
[INFO] '질량 증가 각인서' 시세 데이터 조회 중...
[SUCCESS] '질량 증가 각인서' 데이터 저장 완료!
[INFO] '마나의 흐름 각인서' 시세 데이터 조회 중...
[SUCCESS] '마나의 흐름 각인서' 데이터 저장 완료!
[INFO] '슈퍼 차지 각인서' 시세 데이터 조회 중...
[SUCCESS] '슈퍼 차지 각인서' 데이터 저장 완료!
[INFO] 모든 데이터가 데이터베이스에 저장되었습니다.
[INFO] 데이터를 CSV 파일로 저장했습니다: item_price_data.csv
[START] 그래프 생성 및 표시 작업 시작
[INF

[INFO] 1시간 대기 시작...

[START] 데이터 업데이트 작업 시작
[INFO] item_prices 테이블 확인 및 생성 완료.
[INFO] DB에서 조회된 아이템 이름: ['아드레날린 각인서', '원한 각인서', '예리한 둔기 각인서', '돌격대장 각인서', '저주받은 인형 각인서', '기습의 대가 각인서', '타격의 대가 각인서', '질량 증가 각인서', '마나의 흐름 각인서', '슈퍼 차지 각인서']
[INFO] '아드레날린 각인서' 시세 데이터 조회 중...
[SUCCESS] '아드레날린 각인서' 데이터 저장 완료!
[INFO] '원한 각인서' 시세 데이터 조회 중...
[SUCCESS] '원한 각인서' 데이터 저장 완료!
[INFO] '예리한 둔기 각인서' 시세 데이터 조회 중...
[SUCCESS] '예리한 둔기 각인서' 데이터 저장 완료!
[INFO] '돌격대장 각인서' 시세 데이터 조회 중...
[SUCCESS] '돌격대장 각인서' 데이터 저장 완료!
[INFO] '저주받은 인형 각인서' 시세 데이터 조회 중...
[SUCCESS] '저주받은 인형 각인서' 데이터 저장 완료!
[INFO] '기습의 대가 각인서' 시세 데이터 조회 중...
[SUCCESS] '기습의 대가 각인서' 데이터 저장 완료!
[INFO] '타격의 대가 각인서' 시세 데이터 조회 중...
[SUCCESS] '타격의 대가 각인서' 데이터 저장 완료!
[INFO] '질량 증가 각인서' 시세 데이터 조회 중...
[SUCCESS] '질량 증가 각인서' 데이터 저장 완료!
[INFO] '마나의 흐름 각인서' 시세 데이터 조회 중...
[SUCCESS] '마나의 흐름 각인서' 데이터 저장 완료!
[INFO] '슈퍼 차지 각인서' 시세 데이터 조회 중...
[SUCCESS] '슈퍼 차지 각인서' 데이터 저장 완료!
[INFO] 모든 데이터가 데이터베이스에 저장되었습니다.
[INFO] 데이터를 CSV 파일로 저장했습니다: item_price_data.csv
[START] 그래

[INFO] 1시간 대기 시작...

[START] 데이터 업데이트 작업 시작
[INFO] item_prices 테이블 확인 및 생성 완료.
[INFO] DB에서 조회된 아이템 이름: ['아드레날린 각인서', '원한 각인서', '예리한 둔기 각인서', '돌격대장 각인서', '저주받은 인형 각인서', '기습의 대가 각인서', '타격의 대가 각인서', '질량 증가 각인서', '마나의 흐름 각인서', '슈퍼 차지 각인서']
[INFO] '아드레날린 각인서' 시세 데이터 조회 중...
[SUCCESS] '아드레날린 각인서' 데이터 저장 완료!
[INFO] '원한 각인서' 시세 데이터 조회 중...
[SUCCESS] '원한 각인서' 데이터 저장 완료!
[INFO] '예리한 둔기 각인서' 시세 데이터 조회 중...
[SUCCESS] '예리한 둔기 각인서' 데이터 저장 완료!
[INFO] '돌격대장 각인서' 시세 데이터 조회 중...
[SUCCESS] '돌격대장 각인서' 데이터 저장 완료!
[INFO] '저주받은 인형 각인서' 시세 데이터 조회 중...
[SUCCESS] '저주받은 인형 각인서' 데이터 저장 완료!
[INFO] '기습의 대가 각인서' 시세 데이터 조회 중...
[SUCCESS] '기습의 대가 각인서' 데이터 저장 완료!
[INFO] '타격의 대가 각인서' 시세 데이터 조회 중...
[SUCCESS] '타격의 대가 각인서' 데이터 저장 완료!
[INFO] '질량 증가 각인서' 시세 데이터 조회 중...
[SUCCESS] '질량 증가 각인서' 데이터 저장 완료!
[INFO] '마나의 흐름 각인서' 시세 데이터 조회 중...
[SUCCESS] '마나의 흐름 각인서' 데이터 저장 완료!
[INFO] '슈퍼 차지 각인서' 시세 데이터 조회 중...
[SUCCESS] '슈퍼 차지 각인서' 데이터 저장 완료!
[INFO] 모든 데이터가 데이터베이스에 저장되었습니다.
[INFO] 데이터를 CSV 파일로 저장했습니다: item_price_data.csv
[START] 그래

[INFO] 1시간 대기 시작...

[START] 데이터 업데이트 작업 시작
[INFO] item_prices 테이블 확인 및 생성 완료.
[INFO] DB에서 조회된 아이템 이름: ['아드레날린 각인서', '원한 각인서', '예리한 둔기 각인서', '돌격대장 각인서', '저주받은 인형 각인서', '기습의 대가 각인서', '타격의 대가 각인서', '질량 증가 각인서', '마나의 흐름 각인서', '슈퍼 차지 각인서']
[INFO] '아드레날린 각인서' 시세 데이터 조회 중...
[SUCCESS] '아드레날린 각인서' 데이터 저장 완료!
[INFO] '원한 각인서' 시세 데이터 조회 중...
[SUCCESS] '원한 각인서' 데이터 저장 완료!
[INFO] '예리한 둔기 각인서' 시세 데이터 조회 중...
[SUCCESS] '예리한 둔기 각인서' 데이터 저장 완료!
[INFO] '돌격대장 각인서' 시세 데이터 조회 중...
[SUCCESS] '돌격대장 각인서' 데이터 저장 완료!
[INFO] '저주받은 인형 각인서' 시세 데이터 조회 중...
[SUCCESS] '저주받은 인형 각인서' 데이터 저장 완료!
[INFO] '기습의 대가 각인서' 시세 데이터 조회 중...
[SUCCESS] '기습의 대가 각인서' 데이터 저장 완료!
[INFO] '타격의 대가 각인서' 시세 데이터 조회 중...
[SUCCESS] '타격의 대가 각인서' 데이터 저장 완료!
[INFO] '질량 증가 각인서' 시세 데이터 조회 중...
[SUCCESS] '질량 증가 각인서' 데이터 저장 완료!
[INFO] '마나의 흐름 각인서' 시세 데이터 조회 중...
[SUCCESS] '마나의 흐름 각인서' 데이터 저장 완료!
[INFO] '슈퍼 차지 각인서' 시세 데이터 조회 중...
[SUCCESS] '슈퍼 차지 각인서' 데이터 저장 완료!
[INFO] 모든 데이터가 데이터베이스에 저장되었습니다.
[INFO] 데이터를 CSV 파일로 저장했습니다: item_price_data.csv
[START] 그래

[INFO] 1시간 대기 시작...
